In [1]:
!pip install lightgbm
!pip install xgboost
!pip install catboost
!pip install tqdm
!pip install sktime

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 1.4/1.4 MB 6.1 MB/s eta 0:00:00
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 101.1/101.1 MB 6.7 MB/s eta 0:00:00
     ---------------------------------------- 47.0/47.0 kB ? eta 0:00:00
     ---------------------------------------- 15.6/15.6 MB 9.8 MB/s eta 0:00:00
  Using cached tenacity-8.2.3-py3-none-any.whl (24 kB)
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [81]:
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from utils.util import clean_column_names, get_rscore
from scipy import stats
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
%reload_ext autotime

time: 0 ns (started: 2024-06-26 17:16:21 +09:00)


In [12]:
org = pd.read_csv("../data/electric_train.csv")
test = pd.read_csv("../data/electric_test.csv")

train = org.copy()

time: 12.1 s (started: 2024-06-26 16:38:08 +09:00)


In [13]:
test.columns

Index(['electric_test.num', 'electric_test.tm', 'electric_test.hh24',
       'electric_test.stn', 'electric_test.nph_ta', 'electric_test.nph_hm',
       'electric_test.nph_ws_10m', 'electric_test.nph_rn_60m',
       'electric_test.nph_ta_chi', 'electric_test.weekday',
       'electric_test.week_name'],
      dtype='object')

time: 16 ms (started: 2024-06-26 16:38:20 +09:00)


In [16]:
train['electric_train.tm']

0          2021-01-01 01:00:00
1          2021-01-01 02:00:00
2          2021-01-01 03:00:00
3          2021-01-01 04:00:00
4          2021-01-01 05:00:00
                  ...         
7593350    2022-12-31 20:00:00
7593351    2022-12-31 21:00:00
7593352    2022-12-31 22:00:00
7593353    2022-12-31 23:00:00
7593354    2023-01-01 00:00:00
Name: electric_train.tm, Length: 7593355, dtype: object

time: 16 ms (started: 2024-06-26 16:39:12 +09:00)


In [19]:
def add_date_features(df: pd.DataFrame, date_column: str, prefix: str) -> pd.DataFrame:
    # Ensure the date column is in datetime format
    if not pd.api.types.is_datetime64_any_dtype(df[date_column]):
        df[date_column] = pd.to_datetime(df[date_column])
        
    # Adding date features
    df[f'{prefix}.year'] = df[date_column].dt.year
    df[f'{prefix}.month'] = df[date_column].dt.month
    df[f'{prefix}.day'] = df[date_column].dt.day
    df[f'{prefix}.quarter'] = df[date_column].dt.quarter
    df[f'{prefix}.week'] = df[date_column].dt.isocalendar().week.astype(np.int32)
    
    return df

train = add_date_features(train, 'electric_train.tm', 'electric_train')
test = add_date_features(test, 'electric_test.tm', 'electric_test')

time: 2.58 s (started: 2024-06-26 16:41:01 +09:00)


In [22]:
outlier_base = 7

# 전력기상지수의 이상치 탐지 (예제로 Z-score 방법 사용)
z_scores = np.abs(stats.zscore(train['electric_train.elec']))
outliers = train[z_scores > outlier_base]

# 지점번호별 월 평균 전력기상지수 계산
mean_by_station_and_month = train.groupby(['electric_train.stn', 'electric_train.month'])['electric_train.elec'].mean().reset_index()
mean_by_station_and_month.rename(columns={'electric_train.elec': 'electric_train.elec_month'}, inplace=True)

# 원본 데이터와 월별 평균값을 결합 (merge)
train = train.reset_index()
train = pd.merge(train, mean_by_station_and_month, on=['electric_train.stn', 'electric_train.month'], how='left')

# 이상치를 월별 평균값으로 대체
train['electric_train.elec'] = np.where(z_scores > outlier_base, train['electric_train.elec_month'], train['electric_train.elec'])

# 필요에 따라 사용된 임시 열 제거
train.drop(columns=['electric_train.elec_month','index'], inplace=True)

time: 5.56 s (started: 2024-06-26 16:42:05 +09:00)


In [26]:
def show_all_columns():
    """Set pandas display options to show all columns."""
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.expand_frame_repr', False)  # Prevent column wrapping
    pd.set_option('display.max_colwidth', None)  # Show full column content

time: 0 ns (started: 2024-06-26 16:44:15 +09:00)


In [28]:
show_all_columns()

time: 0 ns (started: 2024-06-26 16:44:23 +09:00)


In [29]:
train

,electric_train.num,electric_train.tm,electric_train.hh24,electric_train.n,electric_train.stn,electric_train.sum_qctr,electric_train.sum_load,electric_train.n_mean_load,electric_train.nph_ta,electric_train.nph_hm,electric_train.nph_ws_10m,electric_train.nph_rn_60m,electric_train.nph_ta_chi,electric_train.weekday,electric_train.week_name,electric_train.elec,electric_train.year,electric_train.month,electric_train.day,electric_train.quarter,electric_train.week
0,4821,2021-01-01 01:00:00,1,11,884,6950,751.32,68.606449,2.2,62.7,1.8,0.0,-1.0,4,0,99.56,2021,1,1,1,53
1,4821,2021-01-01 02:00:00,2,11,884,6950,692.60,68.606449,2.3,63.1,2.1,0.0,-0.6,4,0,91.78,2021,1,1,1,53
2,4821,2021-01-01 03:00:00,3,11,884,6950,597.48,68.606449,2.2,62.4,2.5,0.0,-1.3,4,0,79.17,2021,1,1,1,53
3,4821,2021-01-01 04:00:00,4,11,884,6950,553.48,68.606449,1.7,63.5,1.7,0.0,-0.2,4,0,73.34,2021,1,1,1,53
4,4821,2021-01-01 05:00:00,5,11,884,6950,526.24,68.606449,1.7,63.0,1.6,0.0,-0.8,4,0,69.73,2021,1,1,1,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7593350,20947,2022-12-31 20:00:00,20,23,671,34200,6779.84,225.461986,2.7,46.3,3.1,0.0,-0.4,5,1,130.74,2022,12,31,4,52
7593351,20947,2022-12-31 21:00:00,21,23,671,34200,6802.40,225.461986,2.6,46.8,3.1,0.0,-0.5,5,1,131.18,2022,12,31,4,52
7593352,20947,2022-12-31 22:00:00,22,23,671,34200,6706.68,225.461986,2.4,47.4,2.1,0.0,0.2,5,1,129.33,2022,12,31,4,52
7593353,20947,2022-12-31 23:00:00,23,23,671,34200,6355.88,225.461986,2.5,47.0,2.1,0.0,0.3,5,1,122.57,2022,12,31,4,52


time: 16 ms (started: 2024-06-26 16:44:24 +09:00)


In [34]:
def add_time_features(df: pd.DataFrame, hour_column: str, prefix: str = 'hour') -> pd.DataFrame:
    df[f'{prefix}_sin_time'] = np.sin(2 * np.pi * df[hour_column] / 24)
    df[f'{prefix}_cos_time'] = np.cos(2 * np.pi * df[hour_column] / 24)
    return df

time: 0 ns (started: 2024-06-26 16:48:35 +09:00)


In [36]:
train = add_time_features(train, 'electric_train.hh24', 'hour')
test = add_time_features(test, 'electric_test.hh24', 'hour')

time: 219 ms (started: 2024-06-26 16:48:47 +09:00)


In [44]:
def calculate_thi(df: pd.DataFrame, temp_colum: str, humidity_column: str) -> pd.DataFrame:
    df['THI'] = 1.8 * df[temp_colum] - 0.55 * (1 - (df[humidity_column] / 100)) * (1.8 * df[temp_colum] - 26) + 32
    return df

time: 0 ns (started: 2024-06-26 16:56:19 +09:00)


In [45]:
train = calculate_thi(train, 'electric_train.nph_ta', 'electric_train.nph_hm')
test = calculate_thi(test, 'electric_test.nph_ta', 'electric_test.nph_hm')

time: 156 ms (started: 2024-06-26 16:56:20 +09:00)


In [48]:
def calculate_cdd(df: pd.DataFrame, temp_column: str, base_temp: float = 24.0) -> pd.DataFrame:
    df['CDD'] = df[temp_column].apply(lambda x: max(0, x - base_temp))
    return df

time: 0 ns (started: 2024-06-26 16:58:34 +09:00)


In [49]:
train = calculate_cdd(train, 'electric_train.nph_ta')
test = calculate_cdd(test, 'electric_test.nph_ta')

time: 3.44 s (started: 2024-06-26 16:59:18 +09:00)


In [53]:
train['CDD'].value_counts()

0.0     6474512
0.1       28929
0.2       28496
0.3       28475
0.4       27832
         ...   
12.8          7
12.7          7
12.4          5
12.9          4
12.5          1
Name: CDD, Length: 130, dtype: int64

time: 62 ms (started: 2024-06-26 17:00:13 +09:00)


In [55]:
train_df = train.copy()
test_df = test.copy()

time: 1.14 s (started: 2024-06-26 17:03:04 +09:00)


In [56]:
train_df

,electric_train.num,electric_train.tm,electric_train.hh24,electric_train.n,electric_train.stn,electric_train.sum_qctr,electric_train.sum_load,electric_train.n_mean_load,electric_train.nph_ta,electric_train.nph_hm,electric_train.nph_ws_10m,electric_train.nph_rn_60m,electric_train.nph_ta_chi,electric_train.weekday,electric_train.week_name,electric_train.elec,electric_train.year,electric_train.month,electric_train.day,electric_train.quarter,electric_train.week,hour_sin_time,hour_cos_time,THI,CDD
0,4821,2021-01-01 01:00:00,1,11,884,6950,751.32,68.606449,2.2,62.7,1.8,0.0,-1.0,4,0,99.56,2021,1,1,1,53,2.588190e-01,0.965926,40.481506,0.0
1,4821,2021-01-01 02:00:00,2,11,884,6950,692.60,68.606449,2.3,63.1,2.1,0.0,-0.6,4,0,91.78,2021,1,1,1,53,5.000000e-01,0.866025,40.576487,0.0
2,4821,2021-01-01 03:00:00,3,11,884,6950,597.48,68.606449,2.2,62.4,2.5,0.0,-1.3,4,0,79.17,2021,1,1,1,53,7.071068e-01,0.707107,40.517872,0.0
3,4821,2021-01-01 04:00:00,4,11,884,6950,553.48,68.606449,1.7,63.5,1.7,0.0,-0.2,4,0,73.34,2021,1,1,1,53,8.660254e-01,0.500000,39.665205,0.0
4,4821,2021-01-01 05:00:00,5,11,884,6950,526.24,68.606449,1.7,63.0,1.6,0.0,-0.8,4,0,69.73,2021,1,1,1,53,9.659258e-01,0.258819,39.728290,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7593350,20947,2022-12-31 20:00:00,20,23,671,34200,6779.84,225.461986,2.7,46.3,3.1,0.0,-0.4,5,1,130.74,2022,12,31,4,52,-8.660254e-01,0.500000,43.103699,0.0
7593351,20947,2022-12-31 21:00:00,21,23,671,34200,6802.40,225.461986,2.6,46.8,3.1,0.0,-0.5,5,1,131.18,2022,12,31,4,52,-7.071068e-01,0.707107,42.918232,0.0
7593352,20947,2022-12-31 22:00:00,22,23,671,34200,6706.68,225.461986,2.4,47.4,2.1,0.0,0.2,5,1,129.33,2022,12,31,4,52,-5.000000e-01,0.866025,42.592024,0.0
7593353,20947,2022-12-31 23:00:00,23,23,671,34200,6355.88,225.461986,2.5,47.0,2.1,0.0,0.3,5,1,122.57,2022,12,31,4,52,-2.588190e-01,0.965926,42.767250,0.0


time: 15 ms (started: 2024-06-26 17:03:09 +09:00)


In [57]:
test_df

,electric_test.num,electric_test.tm,electric_test.hh24,electric_test.stn,electric_test.nph_ta,electric_test.nph_hm,electric_test.nph_ws_10m,electric_test.nph_rn_60m,electric_test.nph_ta_chi,electric_test.weekday,electric_test.week_name,electric_test.year,electric_test.month,electric_test.day,electric_test.quarter,electric_test.week,hour_sin_time,hour_cos_time,THI,CDD
0,2385,2023-01-01 01:00:00,1,303,7.8,61.5,6.7,0.0,4.2,6,1,2023,1,1,1,52,2.588190e-01,0.965926,48.572530,0.0
1,2385,2023-01-01 02:00:00,2,303,7.9,60.6,7.6,0.0,4.0,6,1,2023,1,1,1,52,5.000000e-01,0.866025,48.772726,0.0
2,2385,2023-01-01 03:00:00,3,303,8.2,61.9,8.7,0.0,4.1,6,1,2023,1,1,1,52,7.071068e-01,0.707107,49.115342,0.0
3,2385,2023-01-01 04:00:00,4,303,8.4,60.9,9.2,0.0,4.3,6,1,2023,1,1,1,52,8.660254e-01,0.500000,49.459744,0.0
4,2385,2023-01-01 05:00:00,5,303,8.5,60.9,9.5,0.0,4.3,6,1,2023,1,1,1,52,9.659258e-01,0.258819,49.601035,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838234,12322,2023-12-31 20:00:00,20,901,5.6,70.8,3.2,0.0,5.8,6,1,2023,12,31,4,52,-8.660254e-01,0.500000,44.636752,0.0
2838235,12322,2023-12-31 21:00:00,21,901,5.3,69.1,3.6,0.0,4.5,6,1,2023,12,31,4,52,-7.071068e-01,0.707107,44.337377,0.0
2838236,12322,2023-12-31 22:00:00,22,901,5.1,70.6,3.0,0.0,5.1,6,1,2023,12,31,4,52,-5.000000e-01,0.866025,43.899794,0.0
2838237,12322,2023-12-31 23:00:00,23,901,5.2,69.7,3.8,0.0,5.1,6,1,2023,12,31,4,52,-2.588190e-01,0.965926,44.133056,0.0


time: 16 ms (started: 2024-06-26 17:03:10 +09:00)


# train set과 valid set

- train set: 2020년, 2021년
- valid set: 2022년

In [58]:
train_df.set_index('electric_train.tm').sort_index()

,electric_train.num,electric_train.hh24,electric_train.n,electric_train.stn,electric_train.sum_qctr,electric_train.sum_load,electric_train.n_mean_load,electric_train.nph_ta,electric_train.nph_hm,electric_train.nph_ws_10m,electric_train.nph_rn_60m,electric_train.nph_ta_chi,electric_train.weekday,electric_train.week_name,electric_train.elec,electric_train.year,electric_train.month,electric_train.day,electric_train.quarter,electric_train.week,hour_sin_time,hour_cos_time,THI,CDD
electric_train.tm,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01 01:00:00,13615,1,39,140,63850,9732.96,250.490543,-8.5,74.5,0.9,0.0,-5.8,2,0,99.63,2020,1,1,1,1,2.588190e-01,0.965926,22.492325,0.0
2020-01-01 01:00:00,18235,1,20,565,14750,2401.12,116.010234,-8.8,25.6,1.9,0.0,-4.6,2,0,103.49,2020,1,1,1,1,2.588190e-01,0.965926,33.280928,0.0
2020-01-01 01:00:00,18234,1,32,565,71630,12139.16,363.252769,-8.8,25.6,1.9,0.0,-4.6,2,0,104.43,2020,1,1,1,1,2.588190e-01,0.965926,33.280928,0.0
2020-01-01 01:00:00,18233,1,28,512,45840,6723.84,225.924357,-6.3,31.3,2.5,0.0,-7.8,2,0,106.29,2020,1,1,1,1,2.588190e-01,0.965926,34.768919,0.0
2020-01-01 01:00:00,11272,1,44,941,76450,12949.24,287.250770,2.4,68.0,1.2,0.0,-5.3,2,0,102.45,2020,1,1,1,1,2.588190e-01,0.965926,40.135680,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-01 00:00:00,12468,24,40,152,50075,8018.88,187.277655,7.1,60.2,4.5,0.0,-2.3,6,1,107.05,2023,1,1,1,52,-2.449294e-16,1.000000,47.673858,0.0
2023-01-01 00:00:00,16453,24,57,617,85000,13949.76,228.064905,-5.5,81.1,0.2,0.0,-2.9,6,1,107.31,2023,1,1,1,52,-2.449294e-16,1.000000,25.831805,0.0
2023-01-01 00:00:00,11418,24,36,253,63735,11472.28,296.369472,5.6,59.5,1.3,0.0,-1.6,6,1,107.53,2023,1,1,1,52,-2.449294e-16,1.000000,45.626180,0.0


time: 2.31 s (started: 2024-06-26 17:03:39 +09:00)


In [59]:
tmp = train_df.groupby(by='electric_train.stn')['electric_train.tm'].agg(func=['max', 'min'])
tmp['기간'] = tmp['min'].dt.strftime('%Y-%m-%d') + ' ~ ' + tmp['max'].dt.strftime('%Y-%m-%d')

tmp

,max,min,기간
electric_train.stn,,,
98,2023-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2023-01-01
99,2021-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2021-01-01
101,2023-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2023-01-01
104,2023-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2023-01-01
106,2022-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2022-01-01
...,...,...,...
942,2023-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2023-01-01
943,2023-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2023-01-01
950,2023-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2023-01-01


time: 172 ms (started: 2024-06-26 17:03:56 +09:00)


In [60]:
tmp['기간'].value_counts()

2020-01-01 ~ 2023-01-01    158
2020-01-01 ~ 2022-01-01     12
2020-01-01 ~ 2021-01-01      6
2021-01-01 ~ 2022-01-01      5
2021-01-01 ~ 2023-01-01      3
2022-01-01 ~ 2023-01-01      1
Name: 기간, dtype: int64

time: 0 ns (started: 2024-06-26 17:04:07 +09:00)


In [61]:
mask = tmp[tmp['기간'] == '2020-01-01 ~ 2023-01-01'].index

mask

Int64Index([ 98, 101, 104, 108, 112, 114, 119, 127, 129, 131,
            ...
            937, 938, 939, 940, 941, 942, 943, 950, 974, 991],
           dtype='int64', name='electric_train.stn', length=158)

time: 16 ms (started: 2024-06-26 17:04:32 +09:00)


In [62]:
data = train_df[train_df['electric_train.stn'].isin(mask)]
data = data.reset_index(drop=True)

# df1=df1.reset_index(drop=True)

time: 2.64 s (started: 2024-06-26 17:04:54 +09:00)


In [63]:
data

,electric_train.num,electric_train.tm,electric_train.hh24,electric_train.n,electric_train.stn,electric_train.sum_qctr,electric_train.sum_load,electric_train.n_mean_load,electric_train.nph_ta,electric_train.nph_hm,electric_train.nph_ws_10m,electric_train.nph_rn_60m,electric_train.nph_ta_chi,electric_train.weekday,electric_train.week_name,electric_train.elec,electric_train.year,electric_train.month,electric_train.day,electric_train.quarter,electric_train.week,hour_sin_time,hour_cos_time,THI,CDD
0,5565,2020-01-01 01:00:00,1,58,184,42250,4468.72,79.776051,2.3,65.8,1.7,0.0,-0.3,2,0,96.58,2020,1,1,1,1,2.588190e-01,0.965926,40.251866,0.0
1,5565,2020-01-01 02:00:00,2,58,184,42250,4101.00,79.776051,2.5,69.7,1.8,0.0,-0.2,2,0,88.63,2020,1,1,1,1,5.000000e-01,0.866025,40.082975,0.0
2,5565,2020-01-01 03:00:00,3,58,184,42250,3724.60,79.776051,2.8,69.5,2.3,0.0,-0.2,2,0,80.50,2020,1,1,1,1,7.071068e-01,0.707107,40.556040,0.0
3,5565,2020-01-01 04:00:00,4,58,184,42250,3484.40,79.776051,2.8,66.0,1.6,0.0,0.5,2,0,75.31,2020,1,1,1,1,8.660254e-01,0.500000,40.959520,0.0
4,5565,2020-01-01 05:00:00,5,58,184,42250,3409.12,79.776051,3.0,66.1,1.6,0.0,0.5,2,0,73.68,2020,1,1,1,1,9.659258e-01,0.258819,41.240870,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7198670,20947,2022-12-31 20:00:00,20,23,671,34200,6779.84,225.461986,2.7,46.3,3.1,0.0,-0.4,5,1,130.74,2022,12,31,4,52,-8.660254e-01,0.500000,43.103699,0.0
7198671,20947,2022-12-31 21:00:00,21,23,671,34200,6802.40,225.461986,2.6,46.8,3.1,0.0,-0.5,5,1,131.18,2022,12,31,4,52,-7.071068e-01,0.707107,42.918232,0.0
7198672,20947,2022-12-31 22:00:00,22,23,671,34200,6706.68,225.461986,2.4,47.4,2.1,0.0,0.2,5,1,129.33,2022,12,31,4,52,-5.000000e-01,0.866025,42.592024,0.0
7198673,20947,2022-12-31 23:00:00,23,23,671,34200,6355.88,225.461986,2.5,47.0,2.1,0.0,0.3,5,1,122.57,2022,12,31,4,52,-2.588190e-01,0.965926,42.767250,0.0


time: 31 ms (started: 2024-06-26 17:04:58 +09:00)


In [64]:
def clean_column_name_train(df: pd.DataFrame, prefix_to_remove: str='electric_train.') -> pd.DataFrame:
    """
    데이터프레임의 칼럼명에서 접두사를 제거하고, 칼럼명을 매핑에 따라 변경.

    Args:
        df (pd.DataFrame): 원본 데이터프레임

    Returns:
        pd.DataFrame: 칼럼명이 정리된 데이터프레임
    """

    mapping = {
    'tm': 'TM', 'hh24': 'HH24', 'weekday': 'weekday', 'week_name': 'week_name', 'sum_qctr': 'sum_qctr',
    'n': 'n', 'sum_load': 'sum_load', 'n_mean_load': 'n_mean_load', 'elec': 'elec',
    'num': 'NUM', 'stn': 'STN', 'nph_ta': 'nph_ta', 'nph_hm': 'nph_hm', 'nph_ws_10m': 'nph_ws_10m',
    'nph_rn_60m': 'nph_rn_60m', 'nph_ta_chi': 'nph_ta_chi'
    }

    # 접두사 제거
    df.columns = [col.replace(prefix_to_remove, '') for col in df.columns]

    # 칼럼명 변경
    df = df.rename(columns=mapping)

    return df

time: 0 ns (started: 2024-06-26 17:05:40 +09:00)


In [65]:
data = clean_column_name_train(data)

time: 297 ms (started: 2024-06-26 17:05:45 +09:00)


In [66]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7198675 entries, 0 to 7198674
Data columns (total 25 columns):
 #   Column         Dtype         
---  ------         -----         
 0   NUM            int64         
 1   TM             datetime64[ns]
 2   HH24           int64         
 3   n              int64         
 4   STN            int64         
 5   sum_qctr       int64         
 6   sum_load       float64       
 7   n_mean_load    float64       
 8   nph_ta         float64       
 9   nph_hm         float64       
 10  nph_ws_10m     float64       
 11  nph_rn_60m     float64       
 12  nph_ta_chi     float64       
 13  weekday        int64         
 14  week_name      int64         
 15  elec           float64       
 16  year           int64         
 17  month          int64         
 18  day            int64         
 19  quarter        int64         
 20  week           int32         
 21  hour_sin_time  float64       
 22  hour_cos_time  float64       
 23  THI    

In [67]:
data.columns

Index(['NUM', 'TM', 'HH24', 'n', 'STN', 'sum_qctr', 'sum_load', 'n_mean_load',
       'nph_ta', 'nph_hm', 'nph_ws_10m', 'nph_rn_60m', 'nph_ta_chi', 'weekday',
       'week_name', 'elec', 'year', 'month', 'day', 'quarter', 'week',
       'hour_sin_time', 'hour_cos_time', 'THI', 'CDD'],
      dtype='object')

time: 0 ns (started: 2024-06-26 17:05:53 +09:00)


In [74]:
train_data, valid_data = data[data['TM'].dt.year < 2022], data[data['TM'].dt.year >= 2022]

time: 1.16 s (started: 2024-06-26 17:11:32 +09:00)


In [75]:
train_remove_columns = ['elec', 'TM', 'n', 'sum_qctr', 'sum_load', 'n_mean_load']

train_features = [column for column in train_data.columns if column not in train_remove_columns]

time: 0 ns (started: 2024-06-26 17:12:05 +09:00)


In [80]:
train_features

['NUM',
 'HH24',
 'STN',
 'nph_ta',
 'nph_hm',
 'nph_ws_10m',
 'nph_rn_60m',
 'nph_ta_chi',
 'weekday',
 'week_name',
 'year',
 'month',
 'day',
 'quarter',
 'week',
 'hour_sin_time',
 'hour_cos_time',
 'THI',
 'CDD']

time: 0 ns (started: 2024-06-26 17:15:12 +09:00)


In [86]:
X_train = train_data[train_features]
y_train = train_data['elec']
X_valid = valid_data[train_features]
y_valid = valid_data['elec']


categorical_features = ['weekday', 'quarter', 'STN']
numeric_features = [feature for feature in train_features if feature not in categorical_features]

time: 1.83 s (started: 2024-06-26 17:23:37 +09:00)


In [87]:
numeric_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 칼럼 변환기 설정
preprocessor = ColumnTransformer(
    transformers=[
        ('NUM', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# 변환기 적용
X_train = preprocessor.fit_transform(X_train)
X_valid = preprocessor.transform(X_valid)

print(X_train.shape)
print(X_valid.shape)

(4894519, 185)
(2304156, 185)
time: 9.42 s (started: 2024-06-26 17:23:39 +09:00)


In [88]:
cat_params = {'learning_rate': 0.11501,
      'loss_function': 'RMSE',
      'verbose': False,
      'n_estimators': 650,
      'early_stopping_rounds': 100,
      'objective': 'MAE',
      'random_seed': 42,
      }

CAT_model = CatBoostRegressor(**cat_params)
CAT_model.fit(X_train, y_train)

time: 4min 48s (started: 2024-06-26 17:24:00 +09:00)


In [90]:
# 예측
y_pred = CAT_model.predict(X_valid)

# 성능 평가
mse = mean_squared_error(y_valid, y_pred)
r_score = get_rscore(y_valid, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R Score: {r_score}')

Mean Squared Error: 43.044038922204315
R Score: 0.9670317487983385
time: 1.2 s (started: 2024-06-26 17:29:33 +09:00)


# 테스트 제출 코드

In [ ]:
CAT_answer = CAT_model.predict(X_test_dense)

In [ ]:
test['elect'] = CAT_answer

In [ ]:
test=test.drop(columns=['year', 'month','day','quarter','hour_sin_time','hour_cos_time', 'THI','CDH'])

In [ ]:
test.to_csv("240312_0966742_CAT.csv", index=False)